In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os 
os.chdir('/Users/bjarnihaukurbjarnason/Documents/genki-signals')

from genki_signals.buffers import DataBuffer

In [ ]:
import time
from threading import Thread

def run_source(source, buffer):
    source.start()
    while source.is_active:
        buffer.extend(source.read())
        time.sleep(1 / 25)

In [ ]:
from genki_signals.data_sources import CameraDataSource, Sampler

buffer = DataBuffer(maxlen=1)

resolution = (480, 360)

camera = CameraDataSource(resolution=resolution)
source = Sampler({"camera": camera}, sample_rate=29)

t = Thread(target=run_source, args=(source, buffer))
t.start()

In [ ]:
buffer.keys()

In [ ]:
buffer["image"].shape

In [ ]:
buffer.plot("image", plot_type="video")

In [ ]:
source.stop(); t.join()

In [ ]:
import genki_signals.signals as s
from genki_signals.system import System

derived = [
    s.Scale("image", 1/2, "image_halfed"),
    s.Scale("image", 2, "image_doubled"),
    s.Concatenate(["image_halfed", "image", "image_doubled"], "image_concat", axis=1),
]

buffer = DataBuffer(maxlen=1)
system = System(source, derived)

In [ ]:
t = Thread(target=run_source, args=(system, buffer))
t.start()

In [ ]:
buffer.keys()

In [ ]:
buffer.plot("image_halfed", plot_type="video")

In [ ]:
buffer.plot("image_doubled", plot_type="video")

In [ ]:
buffer.plot("image_concat", plot_type="video")

In [ ]:
system.stop(); t.join()